## __🎧 Audio Diarization Project (pyannote.audio)__

#### 🧱 Requisitos del sistema
- Python 3.12.12
- Cuenta en Hugging Face con acceso a modelos pyannote/

## 📌 Descripción
Este proyecto utiliza el pipeline pyannote/speaker-diarization-3.0 para:
- detectar segmentos de voz
- identificar hablantes distintos
- preparar el audio para cortes posteriores

In [ ]:
# 0. Check python version
!python3 --version

In [ ]:
# 1. Limpiar entorno previo
!pip uninstall -y numpy torch torchaudio pyannote.audio

In [ ]:
# 2. Install ffmpeg
!apt-get update
!apt-get install -y ffmpeg

In [ ]:
# 3. Install main libraries
!pip install "numpy<2.0" \
             "torch==2.3.1" \
             "torchvision==0.18.1" \
             "torchaudio==2.3.1" \
             "onnxruntime" \
             "huggingface-hub>=0.19.4" \
             "pyannote.audio==3.1.1" \
             "pydub" \
             "ipywidgets"

# NOTA IMPORTANTE:
# Al terminar, ve al menú arriba: "Entorno de ejecución" -> "Reiniciar sesión".

In [ ]:
import os
import torch

from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import detect_silence

from pyannote.audio import Pipeline
import ipywidgets as widgets

In [ ]:
# Get HF token from Colab userdata
from google.colab import userdata
HF_TOKEN = userdata.get('PYANNOTE_HF_TOKEN')

In [ ]:
# Test diarization pipeline
pipeline = Pipeline.from_pretrained(
   'pyannote/speaker-diarization-3.0',
   use_auth_token=HF_TOKEN
)

_______________
1° Try: Diarization
_______________

In [ ]:
INPUT_DIR = "./audios"
OUTPUT_DIR = "./output"
PRE_CUT_SECONDS = 1.5  # segundos antes de que empiece el niño
MIN_CHILD_SEGMENT = 1.0  # duración mínima para considerar "voz del niño"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# ==========================
# FUNCIÓN PRINCIPAL
# ==========================
def process_audio(audio_path):
    print(f"Procesando: {os.path.basename(audio_path)}")

    # Cargar audio
    audio = AudioSegment.from_wav(audio_path)
    duration = len(audio) / 1000  # segundos

    # Diarización
    diarization = pipeline(audio_path)

    # Extraer segmentos ordenados
    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({
            "speaker": speaker,
            "start": turn.start,
            "end": turn.end,
            "duration": turn.end - turn.start
        })

    segments.sort(key=lambda x: x["start"])

    if len(segments) < 2:
        print("⚠️ No se detectaron suficientes hablantes")
        return None

    # ==========================
    # Siempre se cumple:
    # - Primer hablante = profesor
    # - Segundo hablante largo = niño
    # ==========================
    first_speaker = segments[0]["speaker"]

    child_start = None
    for seg in segments:
        if seg["speaker"] != first_speaker and seg["duration"] >= MIN_CHILD_SEGMENT:
            child_start = seg["start"]
            break

    if child_start is None:
        print("⚠️ No se detectó inicio del niño")
        return None

    # Aplicar margen
    cut_time = max(0, child_start - PRE_CUT_SECONDS)

    # Cortar audio
    cut_audio = audio[int(cut_time * 1000):]

    return cut_audio, cut_time

In [ ]:
!unzip audios.zip

In [ ]:
# ==========================
# PROCESAR CARPETA
# ==========================
for file in os.listdir(INPUT_DIR):
    if not file.lower().endswith(".wav"):
        continue

    input_path = os.path.join(INPUT_DIR, file)
    output_path = os.path.join(
        OUTPUT_DIR,
        file.replace(".wav", "_cut.wav")
    )

    result = process_audio(input_path)

    if result is None:
        continue

    cut_audio, cut_time = result
    cut_audio.export(output_path, format="wav")

    print(f"✔ Guardado: {output_path} (corte en {cut_time:.2f}s)")

______________________
2° Try: VAD
❌ Problemas reales de diarización en estos audios

El profesor y el niño suelen ser clasificados como el mismo speaker
Voces cercanas al micrófono
Duración corta del profesor
Ruido ambiente
Voz infantil → embeddings poco estables
El niño puede empezar a leer sin una pausa clara
El modelo no detecta un “cambio de hablante”
Todo queda como SPEAKER_00
Segmentos del niño aparecen fragmentados
Muchos segmentos cortos < 1.0s
Nunca cumples duration >= MIN_CHILD_SEGMENT
Diarización ≠ detección de “inicio semántico”
Se busca inicio de lectura
El modelo solo sabe: quién habla cuándo
Por lo tanto:

No se busca detectar otro hablante.
Se quiere identificar el momento en que comienza una voz continua distinta al código inicial.
Se utilizará VAD Voice activity detection -> ACTIVIDAD DE VOZ
______________________

In [ ]:
from pyannote.audio import Pipeline
from pydub import AudioSegment
import os

vad = Pipeline.from_pretrained(
    "pyannote/voice-activity-detection",
    use_auth_token=HF_TOKEN
)

MIN_READING_DURATION = 4.0   # segundos continuos leyendo
PRE_CUT_SECONDS = 1.5

INPUT_DIR = "./audios"
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
def process_audio_vad(audio_path):
    audio = AudioSegment.from_wav(audio_path)

    vad_result = vad(audio_path)

    segments = []
    for segment in vad_result.get_timeline():
        segments.append({
            "start": segment.start,
            "end": segment.end,
            "duration": segment.end - segment.start
        })

    # print('##############################################')
    # print(audio_path)
    # print(segments)
    # print('##############################################')

    # Agrupar segmentos cercanos
    merged = []
    for seg in segments:
        if not merged:
            merged.append(seg)
        else:
            last = merged[-1]
            if seg["start"] - last["end"] < 0.4:  # tolerancia silencio corto
                last["end"] = seg["end"]
                last["duration"] = last["end"] - last["start"]
            else:
                merged.append(seg.copy())


    # print('##############################################')
    print(audio_path)c:\Users\Denisse Orellana\Desktop\editar_audio\test._2.json
    print(merged)
    # print('##############################################'

    # Buscar primer bloque largo
    acc = 0.0
    for block in merged:
        acc += block["duration"]
        block["duration_acc"] = acc

        # if block["duration"] >= MIN_READING_DURATION:
        if block["duration_acc"] >= MIN_READING_DURATION:
            cut_time = max(0, block["start"] - PRE_CUT_SECONDS)
            cut_audio = audio[int(cut_time * 1000):]
            return cut_audio, cut_time

    print("⚠️ No se detectó lectura continua")
    return None

In [ ]:
# ==========================
# PROCESAR CARPETA
# ==========================
for file in os.listdir(INPUT_DIR):
    if not file.lower().endswith(".wav"):
        continue

    input_path = os.path.join(INPUT_DIR, file)
    output_path = os.path.join(
        OUTPUT_DIR,
        file.replace(".wav", "_cut.wav")
    )

    result = process_audio_vad(input_path)

    if result is None:
        continue

    cut_audio, cut_time = result
    cut_audio.export(output_path, format="wav")

    print(f"✔ Guardado: {output_path} (corte en {cut_time:.2f}s)")

In [ ]:
# Download zip folder with files
import os
import shutil
from google.colab import files

download_dir = './output'
zip_path = './output.zip'

# Crear un ZIP con todos los archivos
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', download_dir)

# Descargar el ZIP completo
files.download(zip_path)

_________________________
3° Try: VAD + Construcción serie temporal de voz
_________________________

In [ ]:
from pyannote.audio import Pipeline
from pydub import AudioSegment
import os

vad = Pipeline.from_pretrained(
    "pyannote/voice-activity-detection",
    use_auth_token=HF_TOKEN
)

INPUT_DIR = "./audios"
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
WINDOW_SECONDS = 8.0     # ventana máxima de análisis
MIN_TOTAL_VOICE = 4.0   # voz acumulada dentro de la ventana
PRE_CUT_SECONDS = 1.5

In [ ]:
def find_reading_start(segments):
    """
    segments: lista de dicts con start, end, duration
    return: start_time o None
    """

    for i, seg in enumerate(segments):
        window_start = seg["start"]
        window_end = window_start + WINDOW_SECONDS

        total_voice = 0.0

        for s in segments[i:]:
            if s["start"] > window_end:
                break

            overlap_start = max(s["start"], window_start)
            overlap_end = min(s["end"], window_end)

            if overlap_end > overlap_start:
                total_voice += overlap_end - overlap_start

            if total_voice >= MIN_TOTAL_VOICE:
                return window_start

    return None

In [ ]:
# Construir serie temporal de voz
def build_voice_timeline(segments, resolution=0.1):
    timeline = []
    t = 0.0
    end_time = max(s["end"] for s in segments)

    while t < end_time:
        is_voice = any(s["start"] <= t < s["end"] for s in segments)
        timeline.append((t, 1 if is_voice else 0))
        t += resolution

    return timeline

In [ ]:
# Detectar cambio estructural (sin umbral fijo)
import numpy as np

def detect_reading_start_adaptive(timeline, resolution=0.1):
    values = np.array([v for _, v in timeline])

    # promedio móvil acumulado
    cumulative_density = np.cumsum(values) / np.arange(1, len(values)+1)

    # derivada (cambio)
    diff = np.diff(cumulative_density)

    # punto de mayor cambio positivo
    idx = np.argmax(diff)

    return timeline[idx][0]

In [ ]:
def process_audio(audio_path):
    audio = AudioSegment.from_wav(audio_path)
    vad_result = vad(audio_path)

    segments = [{
        "start": s.start,
        "end": s.end
    } for s in vad_result.get_timeline()]

    if not segments:
        return None

    timeline = build_voice_timeline(segments)
    reading_start = detect_reading_start_adaptive(timeline)

    # margen dinámico (proporcional)
    pre_cut = max(0.5, reading_start * 0.15)

    cut_time = max(0, reading_start - pre_cut)
    cut_audio = audio[int(cut_time * 1000):]

    return cut_audio, cut_time


In [ ]:
# ==========================
# PROCESAR CARPETA
# ==========================
for file in os.listdir(INPUT_DIR):
    if not file.lower().endswith(".wav"):
        continue

    input_path = os.path.join(INPUT_DIR, file)
    output_path = os.path.join(
        OUTPUT_DIR,
        file.replace(".wav", "_cut.wav")
    )

    result = process_audio(input_path)

    if result is None:
        continue

    cut_audio, cut_time = result
    cut_audio.export(output_path, format="wav")

    print(f"✔ Guardado: {output_path} (corte en {cut_time:.2f}s)")